In [1]:
from langchain.chains.summarize import load_summarize_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq

In [2]:
# Laoding the document
loader = PyPDFLoader("Documents/Porsche Case File.pdf")
pages = loader.load()

In [ ]:
print(pages)

In [4]:
# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
splits = text_splitter.split_documents(pages)

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

In [6]:
print(len(splits))

81


In [7]:
persist_directory = "chroma_db_summarizer"

In [8]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [9]:
# Prepare the DB
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)
db = Chroma(
    embedding_function=embeddings,
    persist_directory=persist_directory
)


C:\Users\mahil\AppData\Local\Temp\ipykernel_18648\1157268201.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [10]:
# Search the DB
query = "What is the detailed summary of the case?"
docs = db.similarity_search_with_relevance_scores(query, k=3)
print(docs)


[(Document(metadata={'page': 5, 'page_label': '6', 'source': 'Documents/Porsche Case File.pdf'}, page_content='A specific argument was advanced on behalf of the CCL that he was already released on bail\nconsidering all the necessary aspects under the Act of 2015 and his mental, physical and social\nWP-2372-2024.doc health was also impacted in the wake of the incident and his parents are capable\nto care for him and to protect him and they have appointed a security team, outside the house to\nensure his safety.\nThe learned APP focused upon the aspect of amendment of the order, under Section 104 of the Act\nand offered a clarification that the application is not intended to revoke the order passed earlier and\nthe most highlighted aspects of the accident, were placed before the Board. It was also submitted\nthat the wrong act of the CCL had created apprehensions in the mind of common people, who had\nbecome doubtful about their safety, on the public road.\nIt was also argued that after 

In [11]:
# Extract just the documents from the (doc, score) tuples
context_text = "\n\n---\n\n".join([doc[0].page_content for doc in docs])
print(context_text)

A specific argument was advanced on behalf of the CCL that he was already released on bail
considering all the necessary aspects under the Act of 2015 and his mental, physical and social
WP-2372-2024.doc health was also impacted in the wake of the incident and his parents are capable
to care for him and to protect him and they have appointed a security team, outside the house to
ensure his safety.
The learned APP focused upon the aspect of amendment of the order, under Section 104 of the Act
and offered a clarification that the application is not intended to revoke the order passed earlier and
the most highlighted aspects of the accident, were placed before the Board. It was also submitted
that the wrong act of the CCL had created apprehensions in the mind of common people, who had
become doubtful about their safety, on the public road.
It was also argued that after the incident, the CCL became a victim of mob lynching and if released

---

become doubtful about their safety, on the pu

In [15]:
# LLM from Groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7,
)

In [16]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Use the following context to answer the question."
    },
    {
        "role": "user",
        "content": f"Context: {context_text}\n\nQuestion: {query}"
    }
]

In [17]:
response = llm.invoke(messages)
print(response.content)


The case involves a Child in Conflict with Law (CCL) who was released on bail after an incident. The prosecution argued that the CCL's release on bail would pose a moral, physical, and psychological danger to his life due to the risk of mob lynching. The prosecution also highlighted the CCL's alleged negligence and reckless behavior, which led to the incident, and argued that his parents' ability to care for him was questionable.

The CCL's defense argued that he was already released on bail, considering all necessary aspects under the Act of 2015, and that his mental, physical, and social health had been impacted by the incident. The defense also pointed out that the CCL's parents were capable of caring for him and had appointed a security team to ensure his safety.

The court considered the arguments and the fact that the CCL had become a victim of mob lynching after the incident. The court also noted that the CCL's father was arraigned as a co-accused, raising concerns about neglect